# Celastrina

## Celastrina checking
20181212

### check existance of fastq files

```
import pandas as pd
import os
import glob

folderBioHPC = '/work/archive/biophysics/Nick_lab/s185491/Celastrina/fq'
folder_fq = '/archive/butterfly/ready_fastq/'

file_summary = '/home/xcao/w/20181212Celastrina/summary/20181211Celastrina.xlsx'
df_summary = pd.read_excel(file_summary,sheet_name='Sheet1')
sample_prefix = [e[1:] for e in df_summary['ID']]
print('unique sample prefix', len(set(sample_prefix)))

files_fq = glob.glob(folder_fq+'*')
files_use = [e for e in files_fq if os.path.basename(e).split('_')[0] in sample_prefix]

files_possible = []
for e in files_fq:
    if any([os.path.basename(e).startswith(s) for s in sample_prefix]):
        #if e not in files_use:
        files_possible.append(e)

dc_fq=dict([e,[]] for e in sample_prefix)
for e in files_use:
    dc_fq[os.path.basename(e).split('_')[0]].append(e)

from collections import Counter
Counter([len(e) for e in dc_fq.values()])
```

### Check existance of files and see which map files need to be generated

```
import pandas as pd
import os
import glob

folderBioHPC = '/work/archive/biophysics/Nick_lab/s185491/Celastrina/fq'
folder_fq = '/archive/butterfly/ready_fastq/'

file_summary = '/home/xcao/w/20181212Celastrina/summary/20181211Celastrina.xlsx'
df_summary = pd.read_excel(file_summary,sheet_name='Sheet1')
sample_prefix = [e[1:] for e in df_summary['ID']]
print('unique sample prefix', len(set(sample_prefix)))

files_fq = glob.glob(folder_fq+'*')
files_use = [e for e in files_fq if os.path.basename(e).split('_')[0] in sample_prefix]

#commandline = 'scp /archive/butterfly/ready_fastq/{' + ','.join(os.path.basename(e) for e in files_use) +'} s185491@nucleus.biohpc.swmed.edu:/work/archive/biophysics/Nick_lab/s185491/Celastrina/fq/'
#os.system(commandline)
open('/home/xcao/w/20181212Celastrina/summary/20181211Celastrina.fastq','w').write('\n'.join(files_use))

#check existance of map files
files_map = glob.glob('/archive/butterfly/maps/debiased/*.map')
files_map = set([os.path.basename(e) for e in files_map])
print('number of map files',len(files_map))
sample_map = [e for e in sample_prefix if e+'_3913_assembly_v1_snp_step2.map' in files_map]
print(len(sample_map),'map files use')
open('/home/xcao/w/20181212Celastrina/summary/20181211Celastrina.mapfiles','w').write('\n'.join(['/archive/butterfly/maps/debiased/'+e+'_3913_assembly_v1_snp_step2.map' for e in sample_map]))

#check existance of statistics files
files = glob.glob('/archive/butterfly/unbias_pipeline_info/step4_postprocessing/final_stats/*.report')
print(files[:2])
files = set([os.path.basename(e) for e in files])
sample_stats = [e for e in sample_prefix if e+'_3913_assembly_v1_stat.report' in files]
print(len(sample_stats),'file stats')
print('files missing',[e for e in sample_prefix if e not in sample_map])
```

conclusion: '18034A05', '18034A08', '18034A09', '18034A02', '3913' is missing. Run the pipeline.

20181214 Ming Finished Running the pipeline. All files are ready

### map files to npInt8

```
files_map = '/home/xcao/w/20181212Celastrina/summary/20181211Celastrina.mapfiles'
outfolder1 = '/home/xcao/w/20181212Celastrina/summary/map.npInt8.0'
outfolder2 = '/home/xcao/w/20181212Celastrina/summary/map.npInt8.2'

import sys
sys.path.append('/home/xcao/p/xiaolongTools/utils/mapFile')
import mapfileIO
mapfileIO.read2Int8s(filenames=files_map, strand=0, changeGap=True, threads=32, outfolder=outfolder1)
mapfileIO.read2Int8s(filenames=files_map, strand=2, changeGap=True, threads=32, outfolder=outfolder2)
```

### Count gaps in each position

```
import sys
sys.path.append('/home/xcao/p/xiaolongTools/utils/mapFile')
import mapfileIO
import glob
import numpy as np
import pickle

folder = '/home/xcao/w/20181212Celastrina/summary/map.npInt8.0/'
outfile = '/home/xcao/w/20181212Celastrina/summary/map.sitesInfo/gapCount182'
outfileHist = outfile + '.hist'
files = glob.glob(folder+'*')
print('total map files', len(files))

ls_mapInt8 = mapfileIO.loadMapBinaries(files, threads=32)
temp = ls_mapInt8[0]
maplen = len(temp)
gapCount = np.zeros(maplen,dtype=np.int16)
for e in ls_mapInt8:
    gapCount += (e==45)

with open(outfile,'wb') as f:
    pickle.dump(gapCount,f)

gapHist = np.unique(gapCount,return_counts=True)
with open(outfileHist,'w') as f:
    for c, p in zip(*gapHist):
        f.write('{}\t{}\n'.format(c,p))

#average gap per site
gapAvg = gapCount.sum() / maplen
print('average gap per site', gapAvg)
```

It turns out average gap per site for the 182 samples is 36, which is pretty low. Use all sites for genomic phylognetic trees

## Phylogenetic Trees

### make whole genome phylogenetic trees

#### extract all sequences. Use all sites

```
import sys
sys.path.append('/home/xcao/p/xiaolongTools/utils/mapFile')
import mapfileIO
import glob

folder = '/home/xcao/w/20181212Celastrina/summary/map.npInt8.0/'
outfile = '/home/xcao/w/20181212Celastrina/20181512phylo/AllSitesNoFilter/Celastrina182.allsites'
files = glob.glob(folder+'*')
print('total map npInt8 files', len(files))

mapfileIO.npInt8stoFastas(files, threads=32, output=outfile, headerFun=lambda x:x.split('_')[0], npFilter=None, npFilterFun=None, merged=True)
```

transfer to bioHPC ` scp /home/xcao/w/20181212Celastrina/20181512phylo/AllSitesNoFilter/Celastrina182.allsites s185491@nucleus.biohpc.swmed.edu://work/biophysics/s185491/20181216Celastrina/phylo/`

#### split the aligned fasta files to 100 parts

```
python /home2/s185491/p/xiaolongTools/utils/splitFastaAlignments2NPartsContinuingEqualStepOrRandom.py -i /work/biophysics/s185491/20181216Celastrina/phylo/Celastrina182.allsites -o /work/biophysics/s185491/20181216Celastrina/phylo/wholeGenome/100equalStep/ -N 100 -m 1 -t 48
```

#### run ExaML for each part

```
txt_qsub = '''#!/bin/bash
#SBATCH --job-name={n}
#SBATCH --partition=32GB
#SBATCH --nodes=7
#SBATCH --ntasks=224
#SBATCH --time=5-00:00:00
#SBATCH --output=0.%j.out
#SBATCH --mail-user=xiaolong.cao@utsouthwestern.edu
#SBATCH --mail-type=ALL

module load openmpi/intel/3.1.1

python3 /home2/s185491/p/xiaolongTools/utils/runExaML_withFastaForLargeFiles.py -i {filename} -T 224
'''

files = ['/work/biophysics/s185491/20181216Celastrina/phylo/wholeGenome/100equalStep/Celastrina182.allsites.split100.%d'%e for e in range(100)]

import os
for filename in files:
    fout = open('/work/biophysics/s185491/20181216Celastrina/phylo/wholeGenome/qsub/'+os.path.basename(filename),'w')
    fout.write(txt_qsub.format(filename=filename,n=os.path.basename(filename)))
    fout.close()
```

submit jobs. Put job files in one file, and submit through script below. 4 jobs running at the same time
```
nohup python /home2/s185491/p/xiaolongTools/utils/submitJobs.py -N 4 -i /work/biophysics/s185491/20181216Celastrina/phylo/wholeGenome/qsub/Celastrina182.allsites.split100.jobs -k 32GB &
```

#### run ExaML with input starting tree
As the result tree `ExaML_modelFile.Celastrina182.allsites.split100.0/1/2/3` is ready, combine them and use this as a guidance tree to improve the speed.

combine the trees
```
cp /work/biophysics/s185491/20181216Celastrina/phylo/wholeGenome/100equalStep/ExaML_result.Celastrina182.allsites.split100.{0,1,2,3} /work/biophysics/s185491/20181216Celastrina/phylo/wholeGenome/ref/
cat /work/biophysics/s185491/20181216Celastrina/phylo/wholeGenome/ref/ExaML_result.Celastrina182.allsites.split100.{0,1,2,3}>/work/biophysics/s185491/20181216Celastrina/phylo/wholeGenome/ref/split100.0_1_2_3.sum
/home2/s185491/p/anaconda3/anaconda520/bin/sumtrees.py -f 0.00 /work/biophysics/s185491/20181216Celastrina/phylo/wholeGenome/ref/split100.0_1_2_3.sum >/work/biophysics/s185491/20181216Celastrina/phylo/wholeGenome/ref/split100.referenceTree
```

change format to newick (python)
```
import dendropy
from ete3 import Tree
filein = "/work/biophysics/s185491/20181216Celastrina/phylo/wholeGenome/ref/split100.referenceTree"
fileout = filein +'.newick'
mle = dendropy.Tree.get(path=filein, schema="nexus")
tree_txt = mle.as_string(schema='newick')
tree = Tree(tree_txt..split()[1]strip())
tree.resolve_polytomy(recursive=True)
tree.write(outfile=fileout)
```


```
txt_qsub = '''#!/bin/bash
#SBATCH --job-name={n}
#SBATCH --partition=32GB
#SBATCH --nodes=7
#SBATCH --ntasks=224
#SBATCH --time=5-00:00:00
#SBATCH --output=0.%j.out
#SBATCH --mail-user=xiaolong.cao@utsouthwestern.edu
#SBATCH --mail-type=ALL

module load openmpi/intel/3.1.1

python3 /home2/s185491/p/xiaolongTools/utils/runExaML_withFastaForLargeFiles.py -i {filename} -T 224  -r /work/biophysics/s185491/20181216Celastrina/phylo/wholeGenome/ref/split100.referenceTree.newick
'''

files = ['/work/biophysics/s185491/20181216Celastrina/phylo/wholeGenome/100equalStep/Celastrina182.allsites.split100.%d'%e for e in range(100)]

import os
for filename in files:
    fout = open('/work/biophysics/s185491/20181216Celastrina/phylo/wholeGenome/qsub/'+os.path.basename(filename),'w')
    fout.write(txt_qsub.format(filename=filename,n=os.path.basename(filename)))
    fout.close()
```

#### generate a combined tree with 5 trees to have a first look of the result
```
 cat /work/biophysics/s185491/20181216Celastrina/phylo/wholeGenome/100equalStep/ExaML_result.Celastrina182.allsites.split100.{0,2,3,4,8} >/work/biophysics/s185491/20181216Celastrina/phylo/wholeGenome/ref/ExaML_result.Celastrina182.allsites.5trees
 /home2/s185491/p/anaconda3/anaconda520/bin/sumtrees.py -f 0.00 /work/biophysics/s185491/20181216Celastrina/phylo/wholeGenome/ref/ExaML_result.Celastrina182.allsites.5trees >/work/biophysics/s185491/20181216Celastrina/phylo/wholeGenome/ref/ExaML_result.Celastrina182.allsites.5trees.sum
 python /home2/s185491/p/xiaolongTools/utils/trees/sumtreesNEXUS2newick.py -i /work/biophysics/s185491/20181216Celastrina/phylo/wholeGenome/ref/ExaML_result.Celastrina182.allsites.5trees.sum
 /home2/mtang/anaconda2/bin/python /work/biophysics/mtang/SNP_calling/scripts/rename_tree_fullname.py /work/biophysics/s185491/20181216Celastrina/phylo/wholeGenome/ref/ExaML_result.Celastrina182.allsites.5trees.sum.nwk
```

#### combine trees and have a look at whole genome tree

Not all 100 tree finished.

```
 cat /work/biophysics/s185491/20181216Celastrina/phylo/wholeGenome/100equalStep/ExaML_result.Celastrina182.allsites.split100.* >/work/biophysics/s185491/20181216Celastrina/phylo/wholeGenome/ref/ExaML_result.Celastrina182.allsites.46trees
 /home2/s185491/p/anaconda3/anaconda520/bin/sumtrees.py -f 0.00 /work/biophysics/s185491/20181216Celastrina/phylo/wholeGenome/ref/ExaML_result.Celastrina182.allsites.46trees >/work/biophysics/s185491/20181216Celastrina/phylo/wholeGenome/ref/ExaML_result.Celastrina182.allsites.46trees.sum
 python /home2/s185491/p/xiaolongTools/utils/trees/sumtreesNEXUS2newick.py -i /work/biophysics/s185491/20181216Celastrina/phylo/wholeGenome/ref/ExaML_result.Celastrina182.allsites.46trees.sum
 /home2/mtang/anaconda2/bin/python /work/biophysics/mtang/SNP_calling/scripts/rename_tree_fullname.py /work/biophysics/s185491/20181216Celastrina/phylo/wholeGenome/ref/ExaML_result.Celastrina182.allsites.46trees.sum.nwk
```

Astral
```
java -Xmx3000m -jar /home2/s185491/p/Astral/astral.5.6.2.jar -i /work/biophysics/s185491/20181216Celastrina/phylo/wholeGenome/ref/ExaML_result.Celastrina182.allsites.46trees -o /work/biophysics/s185491/20181216Celastrina/phylo/wholeGenome/ref/ExaML_result.Celastrina182.allsites.46trees.astral 
/home2/mtang/anaconda2/bin/python /work/biophysics/mtang/SNP_calling/scripts/rename_tree_fullname.py  /work/biophysics/s185491/20181216Celastrina/phylo/wholeGenome/ref/ExaML_result.Celastrina182.allsites.46trees.astral
```

### identify z-chromsomes

#### run promer3
```
workfolder = '/home/xcao/w/20181212Celastrina/20181216zchromosome/'
target = 'Celastrina3913'
target_genome = '/home/xcao/w/genomes/Celastrina/3913_assembly_v1.fa'
ls_reference = ['Spodopetera', 'Heliconius','Bombyx']
ls_reference_genome=['/home/xcao/w/genomes/Spodoptera_litura/GCF_002706865.1_ASM270686v1_genomic.fna', '/home/xcao/w/genomes/Heliconius_melpomene/Heliconius_melpomene_melpomene_Hmel2.5.scaffolds.fa', '/home/xcao/w/genomes/Bombyx_mori/Bomo_genome_assembly.fa']

commands = '''
cd {workfolder}
/home/xcao/p/mummer/MUMmer3.23/promer  -p {target}2{reference}.promer3 {reference_genome} {target_genome}
/home/xcao/p/mummer/MUMmer3.23/delta-filter -q -r {target}2{reference}.promer3.delta >{target}2{reference}.promer3.delta.qr
/home/xcao/p/mummer/MUMmer3.23/mummerplot {target}2{reference}.promer3.delta.qr -t postscript -p {target}2{reference}.promer3.delta.qr
/home/xcao/p/mummer/MUMmer3.23/show-coords -rcl {target}2{reference}.promer3.delta.qr >{target}{reference}.promer3.coords.qr
'''

for reference, reference_genome in zip(ls_reference, ls_reference_genome):
    print(commands.format(reference=reference,reference_genome=reference_genome,target=target,target_genome=target_genome,workfolder=workfolder))
```

Plotting the scaffold vs chromosome figure. As the scaffolds is short, it is not clear in the figure to see the whole genome alignment. But if zoom in, scaffold were assigned to certain chromsome.

#### assign scaffold of Celastrina according to 'Spodopetera', 'Heliconius','Bombyx'

change `Celastrina3913Bombyx` to `Celastrina3913Heliconius`,`Celastrina3913Spodopetera` for the other two.

```
import pandas as pd
import numpy as np

filename_coord = "/home/xcao/w/20181212Celastrina/20181216zchromosome/Celastrina3913Bombyx.promer3.coords.qr"

filename = filename_coord
l = open(filename).readlines()
ls =[e.split() for e in l[5:]]

df_coord = pd.DataFrame(ls)
df_coord = df_coord[[0,1,3,4,6,7,9,10,11,13,14,16,17,19,20,21,22]].copy()
df_coord[[0,1,3,4,6,7,9,10,11,13,14,16,17,19,20]] = df_coord[[0,1,3,4,6,7,9,10,11,13,14,16,17,19,20]].apply(pd.to_numeric)
df_coord.columns = ['ref_start','ref_end','query_start','query_end','ref_match_len','query_match_len','identity','similarity','percent_stop_codon','ref_len','query_len','ref_coverage','query_coverage','ref_reading_frame','query_reading_frame','ref_id','query_id']
df_coord_group_qr = df_coord.groupby(['query_id','ref_id'])

df_qr = df_coord_group_qr['ref_match_len'].sum()
df_summary = pd.DataFrame([ [e,df_qr[e].idxmax(),df_qr[e].max(),df_qr[e].sum(), df_qr[e].max()/df_qr[e].sum()] for e in df_qr.index.unique(0)])
df_summary.columns = ['test_id','ref_id','scaffold_match_len','total_match_len','ratio_specific']


for n in range(df_summary.shape[0]):
    temp_df = df_coord_group_qr.get_group((df_summary['test_id'][n],df_summary['ref_id'][n]))
    match_fragments = temp_df.shape[0]
    df_summary.loc[n,'match_fragments'] = match_fragments
    positions = []
    for m in range(match_fragments):
        start = temp_df.iloc[m]['ref_start']
        end = temp_df.iloc[m]['ref_end']
        if start > end:
            start, end = end, start
        positions += list(range(start, end))
    position_avg = np.median(positions)
    df_summary.loc[n,'position_avg'] = position_avg
    

df_summary['match_fragments'] = df_summary['match_fragments'].astype(np.int64)
df_summary['position_avg'] = df_summary['position_avg'].astype(np.int64)
df_summary = df_summary.sort_values(by=['ref_id','position_avg'])

df_summary.to_csv('/home/xcao/w/20181212Celastrina/summary/20181217Celastrina3913BombyxChr.csv',sep='\t',index=False)
```

Plotting the scaffold vs chromosome figure. As the scaffolds is short, it is not clear in the figure to see the whole genome alignment. But if zoom in, scaffold were assigned to certain chromsome.

#### get z-chromosome scaffolds according to  'Spodopetera', 'Heliconius','Bombyx'

Bomo_Chr1   Hmel221001o  NC_036217.1 is the z-chromosme of Bombyx, Heliconius, and Spodopetera, respectively.

total matched length with Bombyx: 15315609  
Heliconius: 16508358  
Spodopetera: 16075341

z-chromosome infor based on (scaf num, total length)  
Bombyx: 145    19688914    
Heliconius: 118    20309446    
Spodopetera: 121    19065272    
all: 70    18243563    


use "all" scaffolds.
```
import pandas as pd

filename = '/home/xcao/w/20181212Celastrina/summary/20181217Celastrina3913BombyxChr.csv'
df_bom = pd.read_csv(filename,sep='\t')

filename = '/home/xcao/w/20181212Celastrina/summary/20181217Celastrina3913HeliconiusChr.csv'
df_hel = pd.read_csv(filename,sep='\t')

filename = '/home/xcao/w/20181212Celastrina/summary/20181217Celastrina3913SpodopeteraChr.csv'
df_spo = pd.read_csv(filename,sep='\t')

df_bom_z = df_bom[df_bom['ref_id'] == 'Bomo_Chr1']
df_hel_z = df_hel[df_hel['ref_id'] == 'Hmel221001o']
df_spo_z = df_spo[df_spo['ref_id'] == 'NC_036217.1']

zscf_bom = set(df_bom_z['test_id'])
zscf_hel = set(df_hel_z['test_id'])
zscf_spo = set(df_spo_z['test_id'])
zscf_all = zscf_bom & zscf_hel & zscf_spo

file_genome = '/home/xcao/w/genomes/Celastrina/3913_assembly_v1.fa'
from Bio import SeqIO
fout = open(file_genome+'.head','w')
for _e in SeqIO.parse(file_genome,'fasta'):
    fout.write(_e.description+'\n')
fout.close()
fout = open(file_genome+'.len','w')
for _e in SeqIO.parse(file_genome,'fasta'):
    fout.write(_e.id+'\t'+str(len(_e.seq))+'\n')
fout.close()

dc_scaflen = {}
filename = file_genome+'.len'
for line in open(filename):
    scf,l = line.split()
    l=int(l)
    dc_scaflen[scf] = l

print(sum([dc_scaflen[e] for e in zscf_bom]))
print(sum([dc_scaflen[e] for e in zscf_hel]))
print(sum([dc_scaflen[e] for e in zscf_spo]))
print(sum([dc_scaflen[e] for e in zscf_all]))

open('/home/xcao/w/20181212Celastrina/summary/20181217Celastrina3913_z_scf_basedon3','w').write('\n'.join(zscf_all))

```

#### generate a filter np.array file to extract z-chromosome sequences

npSites is a numpy array with True and False. If True, means a z-chromosome sites based on the three reference species

```
file_genomelen = '/home/xcao/w/genomes/Celastrina/3913_assembly_v1.fa.len'
file_targetscf = '/home/xcao/w/20181212Celastrina/summary/20181217Celastrina3913_z_scf_basedon3'

import numpy as np
import pickle
import itertools
sites = [] #True for keep, False not keep. The same length as the genome
target_scfs = open(file_targetscf).read().split()
target_scfs = set(target_scfs)
for line in open(file_genomelen):
    scf,scflen = line.split()
    scflen = int(scflen)
    if scf in target_scfs:
        sites.append([True] * scflen)
    else:
        sites.append([False] * scflen)
sites = list(itertools.chain.from_iterable(sites))
npSites = np.array(sites)
with open('/home/xcao/w/20181212Celastrina/summary/map.sitesInfo/zChrFilter','wb') as f:
    pickle.dump(npSites,f)
```

### make z-chromosme phylogenetic tree

#### extract sequence

```
import sys
sys.path.append('/home/xcao/p/xiaolongTools/utils/mapFile')
import mapfileIO
import glob

folder = '/home/xcao/w/20181212Celastrina/summary/map.npInt8.0/'
outfile = '/home/xcao/w/20181212Celastrina/20181512phylo/zChrBased3Ref/Celastrina182.zChr'
file_filter = '/home/xcao/w/20181212Celastrina/summary/map.sitesInfo/zChrFilter'
files = glob.glob(folder+'*')
print('total map npInt8 files', len(files))

mapfileIO.npInt8stoFastas(files, threads=32, output=outfile, headerFun=lambda x:x.split('_')[0], npFilter=file_filter, npFilterFun=None, merged=True)
```

transfer to BioHPC
`scp /home/xcao/w/20181212Celastrina/20181512phylo/zChrBased3Ref/Celastrina182.zChr s185491@nucleus.biohpc.swmed.edu:/work/biophysics/s185491/20181216Celastrina/phylo/zChr/`

#### rum ExaML

```
txt_qsub = '''#!/bin/bash
#SBATCH --job-name=zChr
#SBATCH --partition=32GB
#SBATCH --nodes=32
#SBATCH --ntasks=1024
#SBATCH --time=5-00:00:00
#SBATCH --output=0.%j.out
#SBATCH --mail-user=xiaolong.cao@utsouthwestern.edu
#SBATCH --mail-type=ALL

module load openmpi/intel/3.1.1

/home2/s185491/p/anaconda3/anaconda520/bin/python3 /home2/s185491/p/xiaolongTools/utils/runExaML_withFastaForLargeFiles.py -i {filename} -T 1024 -r /work/biophysics/s185491/20181216Celastrina/phylo/wholeGenome/100equalStep/ExaML_result.Celastrina182.allsites.split100.0
'''

files = ['/work/biophysics/s185491/20181216Celastrina/phylo/zChr/Celastrina182.zChr']

import os
for filename in files:
    fout = open('/work/biophysics/s185491/20181216Celastrina/phylo/zChr/qsub.Celastrina182.zChr','w')
    fout.write(txt_qsub.format(filename=filename,n=os.path.basename(filename)))
    fout.close()
```

submit with Jinhui's account.

#### rename trees
```
/home2/mtang/anaconda2/bin/python /work/biophysics/mtang/SNP_calling/scripts/rename_tree_fullname.py /work/biophysics/s185491/20181216Celastrina/phylo/zChr/ExaML_result.Celastrina182.zChr
```

#### run 10 more zChr tree with ExaML, with the output of zChr tree as input
20181219, with Jinhui's account. 
Conclusion: run tree with same input generate almost identical result, through the starting tree and random seed are different.


```
#!/bin/bash
#SBATCH --job-name=zChr
#SBATCH --partition=32GB
#SBATCH --nodes=16
#SBATCH --ntasks=512
#SBATCH --time=5-00:00:00
#SBATCH --output=0.%j.out
#SBATCH --mail-user=xiaolong.cao@utsouthwestern.edu
#SBATCH --mail-type=ALL

module load openmpi/intel/3.1.1
cd /work/biophysics/s185491/20181216Celastrina/phylo/zChr/

mpirun -np 512 /home2/s185491/p/examl/ExaML/examl/examl -m GAMMA -t ExaML_result.Celastrina182.zChr -s Celastrina182.zChr.binary -n Celastrina182.zChr00
mpirun -np 512 /home2/s185491/p/examl/ExaML/examl/examl -m GAMMA -t ExaML_result.Celastrina182.zChr -s Celastrina182.zChr.binary -n Celastrina182.zChr01
mpirun -np 512 /home2/s185491/p/examl/ExaML/examl/examl -m GAMMA -t ExaML_result.Celastrina182.zChr -s Celastrina182.zChr.binary -n Celastrina182.zChr02
mpirun -np 512 /home2/s185491/p/examl/ExaML/examl/examl -m GAMMA -t ExaML_result.Celastrina182.zChr -s Celastrina182.zChr.binary -n Celastrina182.zChr03
mpirun -np 512 /home2/s185491/p/examl/ExaML/examl/examl -m GAMMA -t ExaML_result.Celastrina182.zChr -s Celastrina182.zChr.binary -n Celastrina182.zChr04
mpirun -np 512 /home2/s185491/p/examl/ExaML/examl/examl -m GAMMA -t ExaML_result.Celastrina182.zChr -s Celastrina182.zChr.binary -n Celastrina182.zChr05
mpirun -np 512 /home2/s185491/p/examl/ExaML/examl/examl -m GAMMA -t ExaML_result.Celastrina182.zChr -s Celastrina182.zChr.binary -n Celastrina182.zChr06
mpirun -np 512 /home2/s185491/p/examl/ExaML/examl/examl -m GAMMA -t ExaML_result.Celastrina182.zChr -s Celastrina182.zChr.binary -n Celastrina182.zChr07
mpirun -np 512 /home2/s185491/p/examl/ExaML/examl/examl -m GAMMA -t ExaML_result.Celastrina182.zChr -s Celastrina182.zChr.binary -n Celastrina182.zChr08
mpirun -np 512 /home2/s185491/p/examl/ExaML/examl/examl -m GAMMA -t ExaML_result.Celastrina182.zChr -s Celastrina182.zChr.binary -n Celastrina182.zChr09
```

#### zChr sum of 5 trees


```
 cat /work/biophysics/s185491/20181216Celastrina/phylo/zChr/ExaML_result.Celastrina182.zChr0* >/work/biophysics/s185491/20181216Celastrina/phylo/zChr/ExaML_result.Celastrina182.zChr.all
 /home2/s185491/p/anaconda3/anaconda520/bin/sumtrees.py -f 0.00 /work/biophysics/s185491/20181216Celastrina/phylo/zChr/ExaML_result.Celastrina182.zChr.all >/work/biophysics/s185491/20181216Celastrina/phylo/zChr/ExaML_result.Celastrina182.zChr.sum
 python /home2/s185491/p/xiaolongTools/utils/trees/sumtreesNEXUS2newick.py -i /work/biophysics/s185491/20181216Celastrina/phylo/zChr/ExaML_result.Celastrina182.zChr.sum
 /home2/mtang/anaconda2/bin/python /work/biophysics/mtang/SNP_calling/scripts/rename_tree_fullname.py /work/biophysics/s185491/20181216Celastrina/phylo/zChr/ExaML_result.Celastrina182.zChr.sum.nwk
```

Astral
```
java -Xmx3000m -jar /home2/s185491/p/Astral/astral.5.6.2.jar -i /work/biophysics/s185491/20181216Celastrina/phylo/zChr/ExaML_result.Celastrina182.zChr.all -o /work/biophysics/s185491/20181216Celastrina/phylo/zChr/ExaML_result.Celastrina182.zChr.astral 
/home2/mtang/anaconda2/bin/python /work/biophysics/mtang/SNP_calling/scripts/rename_tree_fullname.py  /work/biophysics/s185491/20181216Celastrina/phylo/zChr/ExaML_result.Celastrina182.zChr.astral
```

### mitoTree

#### get sequences
```
import sys
sys.path.append('/home/xcao/p/xiaolongTools/utils/mapFile')
import mapfileIO
filename = '/home/xcao/w/20181212Celastrina/summary/20181211Celastrina.mapfiles'
ls_files = open(filename).read().split()
files = [f.replace('3913_assembly_v1_snp_step2.map', '3913_mito_snp_step2.map') for f in ls_files]
outfile = '/home/xcao/w/20181212Celastrina/20181512phylo/mitochondria/Celastrina.mito'
mapfileIO.mapFilestoFastas(files, threads=32, output=outfile, headerFun=lambda x:x.split('/')[-1].split('_')[0], npFilter=None, npFilterFun=None, merged=True)
```

#### run MrBayes

cp sequence to bioHPC. For runMrBayes_largeInput, change chain to 2, and number of generation to 20 million
```
#!/bin/bash
#SBATCH --job-name=mito
#SBATCH --partition=256GB
#SBATCH --nodes=1
#SBATCH --ntasks=48
#SBATCH --time=5-00:00:00
#SBATCH --output=0.%j.out
#SBATCH --mail-user=xiaolong.cao@utsouthwestern.edu
#SBATCH --mail-type=ALL

module load openmpi/intel/3.1.1
module load beagle-lib/2.1.2

python /home2/s185491/p/xiaolongTools/project/Junonia/runMrBayes_largeInput.py -i /work/biophysics/s185491/20181216Celastrina/phylo/mitochondria/Celastrina.mito
```

```
cp /work/biophysics/s185491/20181216Celastrina/phylo/mitochondriaNewick/Celastrina.mito /work/biophysics/s185491/20181216Celastrina/phylo/mitochondria/Celastrina.mito.nwk
cp /work/biophysics/s185491/20181216Celastrina/phylo/mitochondriaMrBayes/Celastrina.mito /work/biophysics/s185491/20181216Celastrina/phylo/mitochondria/Celastrina.mito.nex
#rename
/home2/mtang/anaconda2/bin/python /work/biophysics/mtang/SNP_calling/scripts/rename_tree_fullname.py /work/biophysics/s185491/20181216Celastrina/phylo/mitochondria/Celastrina.mito.nwk
```



## PCA

### whole genome alignment 20181216

#### running PCA

run the line below first to load required packages
```
export PATH=$PATH:/home2/wli/local/EIG6.1.1/bin;
module add gsl/1.15;
module add openblas/intel/0.2.14;
```

```
txt = '''
cd /work/biophysics/s185491/20181216Celastrina/PCA/  &&  python /home2/s185491/p/xiaolongTools/WenlinTools.Python3/scripts/fasta2EIGinput.py /work/biophysics/s185491/20181216Celastrina/phylo/{filename}  &&  /home2/wli/local/EIG6.1.1/bin/smartpca.pl -i {filename}.geno -a {filename}.snp -b {filename}.ind -o {filename}.pca -p {filename}.plot -e {filename}.eval -l {filename}.log -snpweightoutname {filename}.snp_weights.txt &
'''
files_sitesKeep = '''Celastrina182.allsites'''
print(txt.format(filename = files_sitesKeep))
```

#### plot PCA, whole genome
20181218
```
filename <- 'C:\\Users\\ATPs\\OneDrive\\Lab\\UTSW\\2018Celastrina\\20181211Celastrina.xlsx'

library(ggplot2)
library(readxl)
library(ggrepel)

mydata <- read_excel(filename,sheet = 'Sheet1')
PCAs = c('Allsites182.PC1',
         'Allsites182.PC2',
         'Allsites182.PC3'
        )

pdf('C:\\Users\\ATPs\\OneDrive\\Lab\\UTSW\\2018Celastrina\\PCA\\Celastrina182.allsites.pca.pdf',width = 30, height = 18)

ggplot(mydata, aes_string(x=PCAs[1],y=PCAs[2])) + geom_point(aes(color=`Taxon name`, size = 5))  + geom_label_repel(aes(label=`DNA number`),label.size=NA,fill=NA) 
ggplot(mydata, aes_string(x=PCAs[1],y=PCAs[3])) + geom_point(aes(color=`Taxon name`, size = 5))  + geom_label_repel(aes(label=`DNA number`),label.size=NA,fill=NA) 
ggplot(mydata, aes_string(x=PCAs[2],y=PCAs[3])) + geom_point(aes(color=`Taxon name`, size = 5))  + geom_label_repel(aes(label=`DNA number`),label.size=NA,fill=NA) 


dev.off()
```

### zChr PCA 20181219

```
files_sitesKeep = '''Celastrina182.zChr'''
folder_fasta = '/work/biophysics/s185491/20181216Celastrina/phylo/zChr'
folder_PCA = '/work/biophysics/s185491/20181216Celastrina/PCA/'
txt = '''
cd {folder_PCA}  &&  python /home2/s185491/p/xiaolongTools/WenlinTools.Python3/scripts/fasta2EIGinput.py {folder}/{filename}  &&  /home2/wli/local/EIG6.1.1/bin/smartpca.pl -i {filename}.geno -a {filename}.snp -b {filename}.ind -o {filename}.pca -p {filename}.plot -e {filename}.eval -l {filename}.log -snpweightoutname {filename}.snp_weights.txt &
'''
print(txt.format(filename = files_sitesKeep,folder=folder,folder_PCA=folder_PCA))
```
run linux
```
export PATH=$PATH:/home2/wli/local/EIG6.1.1/bin;
module add gsl/1.15;
module add openblas/intel/0.2.14;
cd /work/biophysics/s185491/20181216Celastrina/PCA/  &&  python /home2/s185491/p/xiaolongTools/WenlinTools.Python3/scripts/fasta2EIGinput.py /work/biophysics/s185491/20181216Celastrina/phylo/zChr/Celastrina182.zChr  &&  /home2/wli/local/EIG6.1.1/bin/smartpca.pl -i Celastrina182.zChr.geno -a Celastrina182.zChr.snp -b Celastrina182.zChr.ind -o Celastrina182.zChr.pca -p Celastrina182.zChr.plot -e Celastrina182.zChr.eval -l Celastrina182.zChr.log -snpweightoutname Celastrina182.zChr.snp_weights.txt &

```

## STRUCTURE

### zChr

#### convert fasta 2 structure input
```
python /home2/s185491/p/xiaolongTools/utils/STRUCTURE/fasta2STRUCTUREinput_haploid_nt.py -i /work/biophysics/s185491/20181216Celastrina/phylo/zChr/Celastrina182.zChr -o /work/biophysics/s185491/20181216Celastrina/structure/zChr/Celastrina182.zChr -t 48
```
It seems xmfa2struct also output a first line with some position information. 

Note: program is updated to include distance between SNP sites. 182 samples with 924610 SNPs (924611 sites, add first column 0)

the input is too large to run STRUCTURE. split the fasta file to 3 parts
```
python /home2/s185491/p/xiaolongTools/utils/splitFastaAlignments2NPartsContinuingEqualStepOrRandom.py -i /work/biophysics/s185491/20181216Celastrina/phylo/zChr/Celastrina182.zChr -t 48 -m 1 -o /work/biophysics/s185491/20181216Celastrina/phylo/zChr/ -N 3```

convert fasta 2 structure input for 3 files
```
python /home2/s185491/p/xiaolongTools/utils/STRUCTURE/fasta2STRUCTUREinput_haploid_nt.py -i /work/biophysics/s185491/20181216Celastrina/phylo/zChr/Celastrina182.zChr.split3.0 -o /work/biophysics/s185491/20181216Celastrina/structure/zChr/Celastrina182.zChr.split3.0 -t 48
python /home2/s185491/p/xiaolongTools/utils/STRUCTURE/fasta2STRUCTUREinput_haploid_nt.py -i /work/biophysics/s185491/20181216Celastrina/phylo/zChr/Celastrina182.zChr.split3.1 -o /work/biophysics/s185491/20181216Celastrina/structure/zChr/Celastrina182.zChr.split3.1 -t 48
python /home2/s185491/p/xiaolongTools/utils/STRUCTURE/fasta2STRUCTUREinput_haploid_nt.py -i /work/biophysics/s185491/20181216Celastrina/phylo/zChr/Celastrina182.zChr.split3.2 -o /work/biophysics/s185491/20181216Celastrina/structure/zChr/Celastrina182.zChr.split3.2 -t 48
```

#### generate cmds and run

182 samples, the sites for split0, 1, and 1 are 307765, 308559, 308289 respectively.  
generate running scripts
```
python /home2/s185491/p/xiaolongTools/utils/STRUCTURE/runSTRUCTUREwithSTRUCTUREinput.py -i /work/biophysics/s185491/20181216Celastrina/structure/zChr/Celastrina182.zChr.split3.0 -I 182 -L 307765 -s 1000 -P 2,3,4,5,6,7,8,9 -o /work/biophysics/s185491/20181216Celastrina/structure/zChr/cmds/jobs.txt
python /home2/s185491/p/xiaolongTools/utils/STRUCTURE/runSTRUCTUREwithSTRUCTUREinput.py -i /work/biophysics/s185491/20181216Celastrina/structure/zChr/Celastrina182.zChr.split3.1 -I 182 -L 308559 -s 1000 -P 2,3,4,5,6,7,8,9 -o /work/biophysics/s185491/20181216Celastrina/structure/zChr/cmds/jobs.txt
python /home2/s185491/p/xiaolongTools/utils/STRUCTURE/runSTRUCTUREwithSTRUCTUREinput.py -i /work/biophysics/s185491/20181216Celastrina/structure/zChr/Celastrina182.zChr.split3.2 -I 182 -L 308289 -s 1000 -P 2,3,4,5,6,7,8,9 -o /work/biophysics/s185491/20181216Celastrina/structure/zChr/cmds/jobs.txt
```
run jobs
```
#!/bin/bash
#SBATCH --job-name=zChr_Structure
#SBATCH --partition=super
#SBATCH --nodes=1
#SBATCH --ntasks=24
#SBATCH --time=10-00:00:00
#SBATCH --output={file_id}.%j.out
#SBATCH --mail-user=xiaolong.cao@utsouthwestern.edu
#SBATCH --mail-type=ALL

python3 /home2/s185491/p/xiaolongTools/multiThread.py 24 /work/biophysics/s185491/20181216Celastrina/structure/zChr/cmds/jobs.txt
```

### whole genome


#### convert fasta 2 structure input

```
txt = '''
python /home2/s185491/p/xiaolongTools/utils/STRUCTURE/fasta2STRUCTUREinput_haploid_nt.py -i /work/biophysics/s185491/20181216Celastrina/phylo/wholeGenome/100equalStep/Celastrina182.allsites.split100.{n} -o /work/biophysics/s185491/20181216Celastrina/structure/whole/Celastrina182.allsites.split100.{n} -t 48'''
for n in range(6):
    print(txt.format(n=n))

```
182 samples. snp counts 249060  249493  249053  249879  249277  249057

#### generate cmds and run
```
snp_counts = '249060 249493 249053 249879 249277 249057'.split()
txt = 'python /home2/s185491/p/xiaolongTools/utils/STRUCTURE/runSTRUCTUREwithSTRUCTUREinput.py -i /work/biophysics/s185491/20181216Celastrina/structure/whole/Celastrina182.allsites.split100.{n} -I 182 -L {snp} -s 1000 -P 2,3,4,5,6,7,8,9 -o /work/biophysics/s185491/20181216Celastrina/structure/whole/cmds/jobs.txt'
for snp,n in zip(snp_counts,range(6)):
    print(txt.format(n=n,snp=snp))
```

```
#!/bin/bash
#SBATCH --job-name=zChr_Structure
#SBATCH --partition=256GB
#SBATCH --nodes=1
#SBATCH --ntasks=48
#SBATCH --time=10-00:00:00
#SBATCH --output={file_id}.%j.out
#SBATCH --mail-user=xiaolong.cao@utsouthwestern.edu
#SBATCH --mail-type=ALL

python3 /home2/s185491/p/xiaolongTools/multiThread.py 48 /work/biophysics/s185491/20181216Celastrina/structure/whole/cmds/jobs.txt
```